In [ ]:
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path
from typing import List

import utm
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import shapely.ops as so

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
cairns = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed = cairns.append_dist_to_stop_times()


In [ ]:
import gtfs_kit.helpers as hp
from typing import Optional, Iterable


def stop_times_to_geojson(feed, trip_ids) -> dict:
    st = (
        feed.stop_times
        .loc[lambda x: x.trip_id.isin(trip_ids)]
        .filter([
            "trip_id",
            "stop_id",
            "stop_sequence",
            "shape_dist_traveled",
            "arrival_time", 
            "departure_time",
        ])
    )
    g = (
        gk.geometrize_stops(feed, stop_ids=st.stop_id.unique())
        .merge(st)
        .sort_values(["trip_id", "stop_sequence"])
        .drop_duplicates(subset=["trip_id", "stop_sequence"])
    )
    if g.empty:
        result = {"type": "FeatureCollection", "features": []}
    else:
        result = hp.drop_feature_ids(json.loads(g.to_json()))

    return result

def trips_to_geojson(
    feed: "Feed",
    trip_ids: Optional[Iterable[str]] = None,
    *,
    include_stops: bool = False,
) -> dict:
    """
    Return a GeoJSON FeatureCollection of LineString features representing the Feed's trips.
    The coordinates reference system is the default one for GeoJSON,
    namely WGS84.

    If ``include_stops``, then include the trip stops as Point features .
    If an iterable of trip IDs is given, then subset to those trips.
    If the subset is empty, then return a FeatureCollection with an empty list of
    features.
    If the Feed has no shapes, then raise a ValueError.
    If any of the given trip IDs are not found in the feed, then raise a ValueError.
    """
    if trip_ids is None:
        return {"type": "FeatureCollection", "features": []}
    
    D = set(trip_ids) - set(feed.trips.trip_id)
    if D:
        raise ValueError(f"Trip IDs {D} not found in feed.")

    # Get trips
    g = geometrize_trips(feed, trip_ids=trip_ids)
    trips_gj = json.loads(g.to_json())

    # Get stops if desired
    if include_stops:
        st_gj = stop_times_to_geojson(feed, trip_ids)
        trips_gj["features"].extend(st_gj["features"])

    return hp.drop_feature_ids(trips_gj)


In [ ]:
ts = feed.compute_trip_stats()
trip_id_1 = ts.loc[lambda x: x.is_loop == 1, "trip_id"].iat[0]
trip_id_2 = feed.trips.trip_id.iat[0]
trip_ids = [trip_id_1, trip_id_2]


In [ ]:
feed.map_trips(trip_ids[:1], include_stops=True, include_arrows=True)

In [ ]:
dates = feed.get_first_week()[6:]
activity = feed.compute_trip_activity(dates)
display(activity)

trip_ids = activity.loc[
    lambda x: x.filter(dates).sum(axis=1) > 0,
    "trip_id",
]
trip_ids

In [ ]:
feed.trips

In [ ]:
dates = feed.get_first_week()[:1] + ["20010101"]

path = DATA_DIR / "cairns_screen_lines.geojson"
screen_lines = gpd.read_file(path)
f = feed.compute_screen_line_counts(screen_lines, dates)
f

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)



In [ ]:
feed = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed.describe()

In [ ]:
shapes = feed.geometrize_shapes(use_utm=True)
shapes.crs
screen_lines = screen_lines.to_crs(shapes.crs)
screen_lines

In [ ]:
url = "http://transitfeeds.com/p/10-15-transit/936/latest/download"
gk.read_feed(url, dist_units="km").describe()

In [ ]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

In [ ]:
feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False)
